In [1]:
import math
import numpy as np
import pandas as pd
from task import Task
from component import Component
from core import Core
from system import System
from simulator import Simulator

## Read csv

In [2]:
large_test_arch_path = "../DRTS_Project-Test-Cases/4-large-test-case/architecture.csv"
large_test_budgets_path = "../DRTS_Project-Test-Cases/4-large-test-case/budgets.csv"
large_test_tasks_path = "../DRTS_Project-Test-Cases/4-large-test-case/tasks.csv"

## Test System class

In [3]:
large_system = System(tasks_file=large_test_tasks_path, arch_file=large_test_arch_path, budgets_file=large_test_budgets_path)
large_system.tasks, large_system.components, large_system.cores

loaded dataframes: 
  core_id  speed_factor scheduler
0  Core_1          0.54       EDF
1  Core_2          0.70       EDF
2  Core_3          0.74        RM
         component_id scheduler  budget  period core_id  priority
0       Camera_Sensor        RM       4      11  Core_1       NaN
1     Image_Processor       EDF       2       7  Core_1       NaN
2    Bitmap_Processor        RM       1       7  Core_1       NaN
3        Lidar_Sensor        RM       1       3  Core_2       NaN
4        Control_Unit       EDF       4       6  Core_2       NaN
5          GPS_Sensor        RM       3      13  Core_3       1.0
6  Communication_Unit        RM       2       4  Core_3       0.0
   task_name  wcet  period        component_id  priority
0     Task_0     4     400       Camera_Sensor       3.0
1     Task_1     8     600       Camera_Sensor       4.0
2     Task_2     3      75       Camera_Sensor       0.0
3     Task_3     2     100       Camera_Sensor       1.0
4     Task_4     2     150     

({'Task_0': <task.Task at 0x7ad088a4abf0>,
  'Task_1': <task.Task at 0x7ad088a4a3e0>,
  'Task_2': <task.Task at 0x7ad088a4ac80>,
  'Task_3': <task.Task at 0x7ad088a4a3b0>,
  'Task_4': <task.Task at 0x7ad088a4a1d0>,
  'Task_5': <task.Task at 0x7ad088a4a380>,
  'Task_6': <task.Task at 0x7ad088a4ac20>,
  'Task_7': <task.Task at 0x7ad088a4a4a0>,
  'Task_8': <task.Task at 0x7ad088a4a950>,
  'Task_9': <task.Task at 0x7ad088a48e80>,
  'Task_10': <task.Task at 0x7ad088a4a860>,
  'Task_11': <task.Task at 0x7ad088a4a650>,
  'Task_12': <task.Task at 0x7ad088a4a5f0>,
  'Task_13': <task.Task at 0x7ad088a4a740>,
  'Task_14': <task.Task at 0x7ad088a4a6e0>,
  'Task_15': <task.Task at 0x7ad088a4a0b0>,
  'Task_16': <task.Task at 0x7ad088a4a170>,
  'Task_17': <task.Task at 0x7ad088a49ea0>,
  'Task_18': <task.Task at 0x7ad088a49e70>,
  'Task_19': <task.Task at 0x7ad088a49ed0>,
  'Task_20': <task.Task at 0x7ad088a4a050>,
  'Task_21': <task.Task at 0x7ad088a4a020>,
  'Task_22': <task.Task at 0x7ad088a49ff0>

In [4]:
for core_name, core in large_system.cores.items():
    print(core_name)
    for component in core.components:
        print(component)

Core_1
Component(id=Camera_Sensor, scheduler=RM, budget=4, period=11, core_id=Core_1, priority=nan, tasks=5, bdr_alpha=0.36, bdr_delta=14.00)
Component(id=Image_Processor, scheduler=EDF, budget=2, period=7, core_id=Core_1, priority=nan, tasks=3, bdr_alpha=0.29, bdr_delta=10.00)
Component(id=Bitmap_Processor, scheduler=RM, budget=1, period=7, core_id=Core_1, priority=nan, tasks=4, bdr_alpha=0.14, bdr_delta=12.00)
Core_2
Component(id=Lidar_Sensor, scheduler=RM, budget=1, period=3, core_id=Core_2, priority=nan, tasks=4, bdr_alpha=0.33, bdr_delta=4.00)
Component(id=Control_Unit, scheduler=EDF, budget=4, period=6, core_id=Core_2, priority=nan, tasks=6, bdr_alpha=0.67, bdr_delta=4.00)
Core_3
Component(id=GPS_Sensor, scheduler=RM, budget=3, period=13, core_id=Core_3, priority=1.0, tasks=2, bdr_alpha=0.23, bdr_delta=20.00)
Component(id=Communication_Unit, scheduler=RM, budget=2, period=4, core_id=Core_3, priority=0.0, tasks=4, bdr_alpha=0.50, bdr_delta=4.00)


In [5]:
for component_name, component in large_system.components.items():
    print(f"{component_name}")
    for task in component.tasks:
        print(task)

Camera_Sensor
task Task_0 with wcet:4, period:400, component_id:Camera_Sensor, priority3.0
task Task_1 with wcet:8, period:600, component_id:Camera_Sensor, priority4.0
task Task_2 with wcet:3, period:75, component_id:Camera_Sensor, priority0.0
task Task_3 with wcet:2, period:100, component_id:Camera_Sensor, priority1.0
task Task_4 with wcet:2, period:150, component_id:Camera_Sensor, priority2.0
Image_Processor
task Task_5 with wcet:2, period:50, component_id:Image_Processor, prioritynan
task Task_6 with wcet:4, period:80, component_id:Image_Processor, prioritynan
task Task_7 with wcet:2, period:200, component_id:Image_Processor, prioritynan
Bitmap_Processor
task Task_8 with wcet:6, period:300, component_id:Bitmap_Processor, priority1.0
task Task_9 with wcet:3, period:600, component_id:Bitmap_Processor, priority2.0
task Task_10 with wcet:3, period:75, component_id:Bitmap_Processor, priority0.0
task Task_11 with wcet:3, period:900, component_id:Bitmap_Processor, priority3.0
Lidar_Sensor


In [6]:
def half_half_algorithm(budget:float, period:float):
    """
    Implements the Half-Half Algorithm for real-time systems.
    This algorithm calculates two parameters, alpha and delta, based on 
    the given budget and period. These parameters can be used for 
    scheduling or resource allocation in real-time systems.
    Parameters:
    -----------
    budget : float
        The allocated execution time or computational budget.
    period : float
        The time period over which the budget is allocated.
    Returns:
    --------
    alpha : represents the utilization factor, calculated as the ratio 
        of budget to period.
    delta : represents a derived parameter, calculated as twice the 
        difference between the period and the budget.
    """
    alpha = budget / period
    delta = 2 * (period - budget)
    return alpha, delta
half_half_algorithm(84, 84)

(1.0, 0)

Makes sense because this task takes up all the processing power with budget = 84 and period = 84

## Test simulator class

In [7]:
large_simulator = Simulator(large_system)
large_simulator.run_simulation()

Running simulation until hyperperiod: 7200
Component Image_Processor: bdr_delta/2 = 5.0, budget = 1.4285714285714284
Component Lidar_Sensor: bdr_delta/2 = 2.0, budget = 0.6666666666666666
Component Communication_Unit: bdr_delta/2 = 2.0, budget = 1.0
Component Lidar_Sensor: available_budget = 0.6666666666666666
Component Lidar_Sensor: Running task Task_13 (instance 0) with remaining_wcet: 12.857142857142858, budget: 0.6666666666666666
Component Communication_Unit: available_budget = 1.0
Component Communication_Unit: Running task Task_27 (instance 0) with remaining_wcet: 4.054054054054054, budget: 1.0
Component Lidar_Sensor: Remaining budget = 0.0
Component Lidar_Sensor: Running task Task_12 (instance 0) with remaining_wcet: 11.428571428571429, budget: 0.0
Component Lidar_Sensor: No budget left
Component Communication_Unit: Remaining budget = 0.0
Component Communication_Unit: Running task Task_24 (instance 0) with remaining_wcet: 4.054054054054054, budget: 0.0
Component Communication_Uni

In [8]:
for component_name, component in large_system.components.items():
    print(f"{component_name}")
    for task in component.tasks:
        print(task.response_times)

Camera_Sensor
[]
[]
[]
[]
[]
Image_Processor
[18.703703703703702, 37.98941798941799, 39.947089947089964, 15.132275132275154, 34.41798941798942, 29.417989417989332, 48.70370370370358, 50.39682539682519, 19.417989417989247, 38.70370370370358, 40.13227513227514, 15.84656084656092, 35.132275132275254, 17.275132275132478, 36.56084656084681, 44.417989417989816, 13.70370370370415, 32.98941798941837, 27.98941798941837, 47.27513227513259, 48.96825396825432, 17.98941798941837, 37.27513227513259, 38.703703703704036, 14.417989417989702, 33.70370370370392, 15.846560846561033, 35.132275132275254, 43.518518518518704, 18.703703703703923, 37.98941798941814, 26.5608465608467, 45.84656084656092, 47.53968253968242, 16.56084656084613, 35.84656084656012, 37.27513227513077, 19.417989417987428, 38.70370370370142, 14.417989417986746, 33.70370370370074, 42.08994708994351, 17.27513227512827, 36.560846560842265, 25.13227513227025, 44.417989417984245, 15.132275132268887, 34.41798941798288, 35.84656084655353, 17.98

In [9]:
large_simulator.report("result_large.csv")

Generated sim results
